In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")


In [ ]:
mnist.train.labels[0]

In [ ]:
mnist.train.images[0][251:260]

In [ ]:
for i in range(100):
    plt.figure()
    plt.imshow(-mnist.train.images[i].reshape(28,28),interpolation='nearest', cmap='gray')
    plt.title(mnist.train.labels[i])

In [ ]:
import pywt

In [ ]:
data = mnist.train.images[0].reshape(28,28)
coeffs = pywt.wavedec2(data, 'haar')
for i in range(1,len(coeffs)):
    print i, coeffs[i][0].shape, coeffs[i][1].shape, coeffs[i][2].shape

In [ ]:
def init_coeffs_mnist():
    MNIST_SHAPE = [(1, 1), (2, 2), (4, 4), (7, 7), (14, 14)]
    coeffs_zero = [np.zeros((2, 2))]
    for h in range(1 ,5):
        coeffs_zero += [[]]   
        s = MNIST_SHAPE[h]
        for k in range(3) :
            coeffs_zero[h] += [np.zeros(s)]        
    return coeffs_zero    

In [ ]:
def add_coeffs_coord(coeffs_ref, coeffs_add, coord):
    v_test = np.array(coeffs_add[0])
    if np.linalg.norm(v_test) < 1e-16:
        coeffs_add[0] = np.copy(coeffs_ref[0])
    for h in range(1,5):
        h_opp = 5 - h
        i = int(coord[0] / 2**h_opp) 
        j = int(coord[1] / 2**h_opp)
        #print i, j
        v_test = np.array([coeffs_add[h][0][i,j], coeffs_add[h][1][i,j], coeffs_add[h][2][i,j]])
        if np.linalg.norm(v_test) < 1e-16:
            for k in range(3) :
                coeffs_add[h][k][i,j] = coeffs_ref[h][k][i,j]
            

In [ ]:
def copy_coeffs_mnist(coeffs):
    coeffs_copy = []
    coeffs_copy += [coeffs[0].copy()]
    for h in range(1,5):
        coeffs_copy += [[]]    
        for k in range(3) :
            coeffs_copy[h] += [np.copy(coeffs[h][k])]
    return coeffs_copy    

In [ ]:
def calcule_path(coord, axes):
    path_i = []
    path_j = []
    for h in range(0,5):
        h_opp = 5 - h
        coord_i = int(coord[0] / 2**h_opp) 
        coord_j = int(coord[1] / 2**h_opp)    
        path_i += [axes[h][coord_i]]
        path_j += [axes[h][coord_j]]
    return path_i, path_j

In [ ]:
def calcule_axes_mnist():
    axes = []
    for h in range(5):
        h_opp = 5 - h
        b_moins = -.5
        b_plus = 27.5
        intervalle = b_plus - b_moins
        coord_max = int(26.5 / 2**h_opp) + 1
        unite = max(intervalle / coord_max, 2**h_opp)
        print unite
        coords = []
        for j in range(coord_max):
            coords += [b_moins + (j+.5) * unite]
        axes += [coords]
    return axes

In [ ]:
def ajoute_coeffs(coeffs, v, h, u ):
    if h == -1 :
        coeffs[0][0,0]= v
    elif h == 0:
        coeffs[0][0,1]= v[0]
        coeffs[0][1,0] = v[1]
        coeffs[0][1,1] = v[2]
    else: 
        coeffs[h][0][u[0]][u[1]] = v[0]
        coeffs[h][1][u[0]][u[1]] = v[1]
        coeffs[h][2][u[0]][u[1]] = v[2] 

def calcule_coeffs_mnist(Data, c, i):
    coeffs = init_coeffs_mnist()
    for h in range(-1,5):
        for u in U[h]:
            v = Data[c][h][u][i]
            ajoute_coeffs(coeffs, v, h, u)
    return coeffs
    
def affiche_mnist(Data, c, i): 
    coeffs_aff = calcule_coeffs_mnist(Data, c, i)
    plt.imshow(-pywt.waverec2(coeffs_aff, 'haar'), interpolation='nearest', cmap='gray')

def affiche_mnist_coord(Data, coeffs_aff, c, i, coord):
    coeffs_ref = calcule_coeffs_mnist(Data, c, i)
    add_coeffs_coord(coeffs_ref, coeffs_aff, coord)
    plt.imshow(-pywt.waverec2(coeffs_aff, 'haar'), interpolation='nearest', cmap='gray')
    '''axes = calcule_axes_mnist()
    path_i, path_j = calcule_path(coord, axes)
    plt.plot(path_j,path_i,'r')
    plt.plot(path_j,path_i,'r+',markersize=12)
    plt.plot(coord[1],coord[0],'+',color='r',markersize=12)
    plt.xlim([b_moins,b_plus])
    plt.ylim([b_moins,b_plus])
    plt.gca().invert_yaxis()'''

def affiche_path_mnist(path_i, path_j):
    b_moins = -.5
    b_plus = 27.5
    plt.plot(path_j,path_i,'r')
    plt.plot(path_j,path_i,'r+',markersize=12)
    plt.xlim([b_moins,b_plus])
    plt.ylim([b_moins,b_plus])
    plt.gca().invert_yaxis()

In [ ]:
coeffs_test = copy_coeffs_mnist(coeffs)
coeffs_zero = init_coeffs_mnist()
        
plt.figure(figsize=(12,8))
plt.subplot(131)
plt.imshow(-pywt.waverec2(coeffs_test, 'haar'), interpolation='nearest', cmap='gray')

for k in range(3) :
    coeffs_test[4][k] *= 0
    coeffs_zero[4][k] = np.copy(coeffs[4][k])
    
plt.subplot(132)
plt.imshow(-pywt.waverec2(coeffs_test, 'haar'), interpolation='nearest', cmap='gray')

plt.subplot(133)
plt.imshow(-pywt.waverec2(coeffs_zero, 'haar'), interpolation='nearest', cmap='gray')

In [ ]:
axes = []
for h in range(0,5):
    h_opp = 5 - h
    b_moins = -.5
    b_plus = 27.5
    intervalle = b_plus - b_moins
    coord_max = int(26.5 / 2**h_opp) + 1
    unite = max(intervalle / coord_max, 2**h_opp)
    print unite
    coords = []
    for j in range(coord_max):
        coords += [b_moins + (j+.5) * unite]
    axes += [coords]
print axes
print ''

U = {-1 : {-1 : -1}}
print 'Niveau -1 :'
print 'U[-1] :', U[-1]
print ''
for h in range(len(axes)):
    #U_ref = {}
    #for pos_i in axes[h]:
    #    for pos_j in axes[h]:
    #        U_ref[h] += [(pos_i, pos_j)]
    U [h] = {}
    for i in range(len(axes[h])):
        for j in range(len(axes[h])):
            U[h][(i,j)] = (axes[h][i], axes[h][j])    
    print 'Niveau ', h, ' : '
    print ' U[' , h, '] :', U[h]
    #print ' U_ref[h] :', U_ref[h]
    print ''
    
bsup = [1,2,4,7,14, 0]

def fils_rec(h,i,j):
    if h < 5 and i < bsup[h] and j < bsup[h] :
        rep = [(h,(i,j)), [], [], [], []]
        rep[1] = fils_rec(h+1,i*2, j*2)
        rep[2] = fils_rec(h+1,i*2, j*2 + 1)
        rep[3] = fils_rec(h+1,i*2 + 1, j*2)
        rep[4] = fils_rec(h+1,i*2 + 1, j*2 + 1)
    else:
        rep = []
    return rep
    
U_tree = fils_rec(0,0,0) 

print U_tree


In [ ]:
#coord = (10.5,10.5)
#coord = (8.5,16.5)
#coord = (16.5,4.5)

t = 1

data = mnist.train.images[t].reshape(28,28)
coeffs = pywt.wavedec2(data, 'haar')

coeffs_coord = init_coeffs_mnist()

for coord in [(10.5,10.5), (8.5,16.5), (16.5,4.5)]:
    add_coeffs_coord(coeffs, coeffs_coord, coord)

    plt.figure(figsize=(20,12))

    path_i, path_j = calcule_path(coord, axes)

    plt.subplot(131)
    plt.imshow(-pywt.waverec2(coeffs, 'haar'), interpolation='nearest', cmap='gray')
    #for u in U_ref:
    #    plt.plot(u[1],u[0],'+',color='b',markersize=12)
    plt.plot(path_j,path_i,'r')
    plt.plot(path_j,path_i,'r+',markersize=12)
    plt.plot(coord[1],coord[0],'+',color='r',markersize=12)
    plt.xlim([b_moins,b_plus])
    plt.ylim([b_moins,b_plus])
    plt.gca().invert_yaxis()

    plt.subplot(132)
    plt.imshow(-pywt.waverec2(coeffs_coord, 'haar'), interpolation='nearest', cmap='gray')
    plt.plot(path_j,path_i,'r')
    plt.plot(path_j,path_i,'r+',markersize=12)
    plt.plot(coord[1],coord[0],'+',color='r',markersize=12)
    plt.xlim([b_moins,b_plus])
    plt.ylim([b_moins,b_plus])
    plt.gca().invert_yaxis()


## Creation de la base d'apprentissage

In [ ]:
len(mnist.train.images)

In [ ]:
B = []
for i in range(len(mnist.train.images)):
    c = mnist.train.labels[i]
    x = mnist.train.images[i].reshape(28,28)
    coeffs = pywt.wavedec2(x, 'haar')
    u_m1 = -1
    v_m1 = coeffs[0][0][0]
    B += [(v_m1,(c, -1, u_m1))]
    u0 = (0,0) #U[0][0]
    v0 = np.array([coeffs[0][0][1], coeffs[0][1][0], coeffs[0][1][1]])
    B += [(v0,(c, 0, u0))]
    for h in range(1,5):
        for u in U[h]:
            #v = np.array([coeffs[h][0][u[1]][u[2]], coeffs[h][1][u[1]][u[2]], coeffs[h][2][u[1]][u[2]]])
            v = np.array([coeffs[h][0][u[0]][u[1]], coeffs[h][1][u[0]][u[1]], coeffs[h][2][u[0]][u[1]]])
            #if np.linalg.norm(v) > 1e-16:
            #B += [(v,(c,u))]
            B += [(v,(c,h,u))]            

## Base de test

In [ ]:
B_test = []
for i in range(len(mnist.test.images)):
    c = mnist.test.labels[i]
    x = mnist.test.images[i].reshape(28,28)
    coeffs = pywt.wavedec2(x, 'haar')
    u_m1 = -1
    v_m1 = coeffs[0][0][0]
    B_test += [(v_m1,(c, -1, u_m1))]
    u0 = (0,0) #U[0][0]
    v0 = np.array([coeffs[0][0][1], coeffs[0][1][0], coeffs[0][1][1]])
    B_test += [(v0,(c, 0, u0))]
    for h in range(1,5):
        for u in U[h]:
            #v = np.array([coeffs[h][0][u[1]][u[2]], coeffs[h][1][u[1]][u[2]], coeffs[h][2][u[1]][u[2]]])
            v = np.array([coeffs[h][0][u[0]][u[1]], coeffs[h][1][u[0]][u[1]], coeffs[h][2][u[0]][u[1]]])
            #if np.linalg.norm(v) > 1e-16:
            #B += [(v,(c,u))]
            B_test += [(v,(c,h,u))]            

## Test images simple

In [ ]:
data_visu = [[],[],[],[],[],[],[],[],[],[]]
for i in range(len(mnist.train.images)):
    c = mnist.train.labels[i]
    x = mnist.train.images[i].reshape(28,28)
    data_visu[c] += [x]


In [ ]:
image_moy_moy = np.zeros((28,28))
for c in range(10):
    image_moy = np.mean(np.array(data_visu[c]),0)
    image_moy_moy += image_moy
    plt.figure(figsize=(12,8))
    plt.subplot(131)
    plt.imshow(-image_moy, interpolation='nearest', cmap='gray')
    plt.title(c)
plt.figure(figsize=(12,8))
plt.subplot(131)
plt.imshow(-image_moy_moy, interpolation='nearest', cmap='gray')
plt.title('all')

## Dictionnaire (Base d'apprentissage)

In [ ]:
Data = [[],[],[],[],[],[],[],[],[],[]]
for c in range(10):
    Data[c] = [{},{},{},{},{}] 
    
for d in B:
    v = d[0]
    c = d[1][0]
    h = d[1][1]
    u = d[1][2]
    if u in Data[c][h]:
        Data[c][h][u] += [v]
    else:
        Data[c][h][u] = [v]


## Dictionnaire (Base de test)

In [ ]:
Data_test = [[],[],[],[],[],[],[],[],[],[]]
for c in range(10):
    Data_test[c] = [{},{},{},{},{}] 
    
for d in B_test:
    v = d[0]
    c = d[1][0]
    h = d[1][1]
    u = d[1][2]
    if u in Data_test[c][h]:
        Data_test[c][h][u] += [v]
    else:
        Data_test[c][h][u] = [v]


In [ ]:
print np.mean(Data[0][0][(0,0)],0)
print np.mean(Data[0][1][(0,0)],0)
print np.mean(Data[0][1][(0,1)],0)
print np.mean(Data[0][2][(0,0)],0)
print np.mean(Data[0][2][(0,1)],0)


In [ ]:
print Data[5][0][(0,0)]

In [ ]:
c = 1
h = -1
i = 0
j = 0
len(Data[c][h][(i,j)])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize = (12,12))
ax = fig.add_subplot(111, projection='3d')

h = 0 #4
i = 0 #5
j = 0 #5

test = np.array(Data[0][h][(i,j)])
ax.scatter(test[:,0], test[:,1], test[:,2],c='r', marker = 'o')

test2 = np.array(Data[1][h][(i,j)])
ax.scatter(test2[:,0], test2[:,1], test2[:,2],c='b', marker = '^')

test3 = np.array(Data[3][h][(i,j)])
ax.scatter(test3[:,0], test3[:,1], test3[:,2],c='g', marker = 'd')

ax.view_init(20, -30)

'''for angle in range(0, 360):
    ax.view_init(30, angle)
    plt.draw()
    plt.pause(.001)'''

## Probas elementaires 

In [ ]:
mu = []
Sigma = []
rho = []
for c in range(10):
    mu += [{}]
    Sigma += [{}]
    rho += [{}]
    for h in range(-1,5):
        mu[c][h] = {}
        Sigma[c][h] = {}
        rho[c][h] = {}
        for u in U[h]:
            if u in Data[c][h]:
                data = []
                cpt = 0
                for v in Data[c][h][u]:
                    if np.linalg.norm(v) < 1e-16:
                        cpt += 1
                    else:
                        data += [v]
                if h == -1 :
                    print len(data)
                if len(data) > 1:
                    mu[c][h][u] = np.mean(data, 0) #Data[c][h][u],0)
                    Sigma[c][h][u] = np.cov(np.array(data).T) #Data[c][h][u]).T)
                    rho[c][h][u] = float(cpt) / len(Data[c][h][u])
                else:
                    mu[c][h][u] = np.zeros(3)
                    Sigma[c][h][u] = np.zeros((3,3))
                    rho[c][h][u] = 1.
            

In [ ]:
c = 0
h = -1 # 4
u = -1 #(12,12)
print mu[c][h][u]
print Sigma[c][h][u]
print rho[c][h][u]


In [ ]:
mu[0][-1][-1]

## Reconstruction


In [ ]:
for c in range(10):
    coeffs_zero = init_coeffs_mnist()
    coeffs_zero[0][0,0]= (1 - rho[c][-1][-1]) * mu[c][-1][-1]
    coeffs_zero[0][0,1]= (1 - rho[c][0][(0,0)]) * mu[c][0][(0,0)][0]
    coeffs_zero[0][1,0] = (1 - rho[c][0][(0,0)]) * mu[c][0][(0,0)][1]
    coeffs_zero[0][1,1] = (1 - rho[c][0][(0,0)]) * mu[c][0][(0,0)][2]
    for h in range(1,5):
        for u in U[h]:
            coeffs_zero[h][0][u[0]][u[1]] = (1 - rho[c][h][u]) * mu[c][h][u][0]
            coeffs_zero[h][1][u[0]][u[1]] = (1 - rho[c][h][u]) * mu[c][h][u][1]
            coeffs_zero[h][2][u[0]][u[1]] = (1 - rho[c][h][u]) * mu[c][h][u][2]

    plt.figure(figsize=(12,8))
    plt.subplot(131)
    plt.imshow(-pywt.waverec2(coeffs_zero, 'haar'), interpolation='nearest', cmap='gray')



## Test distribution

In [ ]:
fig = plt.figure(figsize = (12,12))
ax = fig.add_subplot(111, projection='3d')

c = 8
h = 3
u = (4,1)

n = 200

test2 = np.random.multivariate_normal(mu[c][h][u], Sigma[c][h][u] , n)
#test2 = np.random.multivariate_normal(mu[(c,h,u)], Sig_test * 0 , n)
#test2 = np.random.multivariate_normal(-mu[(c,h,u)], np.eye(3,3), n)
ax.scatter(test2[:,0], test2[:,1], test2[:,2],c='b', marker = '^')

#test3 = np.random.multivariate_normal(-mu[(c,h,u)], np.eye(3,3), n)
#ax.scatter(test3[:,0], test3[:,1], test3[:,2],c='g', marker = 'd')

index = np.arange(len(Data[c][h][u]))
np.random.shuffle(index)
test = np.array(Data[c][h][u])
ax.scatter(test[index[:n],0], test[index[:n],1], test[index[:n],2],c='r', marker = 'o')

v = Data_test[c][h][u][0]
ax.scatter(v[0], v[1], v[2],c='g', marker = 's')

ax.view_init(20, -30)

## Test proba

In [ ]:
from scipy.stats import multivariate_normal

c_ref = 0
#h = 0
#u = (0,0)

for i in [0]:
    pi = np.ones(10) / 10
    log_score = np.zeros(10)
    mem_pi = [pi]
    for h in range(-1,5):
        print '*************************'
        print '*********' , 'h = ' , h , '************'
        print '*************************'
        for u in U[h]:
            #print U[h]
            #u = (u_ref[1],u_ref[2])
            lik = np.zeros(10)
            for c in range(10):
                v = Data_test[c_ref][h][u][i]
                if np.linalg.norm(v) < 1e-16:
                    lik[c] = rho[c][h][u]
                else:
                    if np.linalg.norm(mu[c][h][u]) > 1e-16:
                        dist = multivariate_normal(mean = mu[c][h][u], cov = Sigma[c][h][u])
                        #print 'mean =', mu[c][h][u]
                        #print 'cov =', Sigma[c][h][u]
                        lik[c] = (1-rho[c][h][u]) * dist.pdf(v)
                    else:
                        lik[c] = 0
                lik[c] = max(lik[c],1e-16)
            print 'lik =' + str(lik) 
            log_score += np.log(lik) 
            max_log_score = max(log_score)
            print 'log_score', log_score
            print 'max(log_score) :', max_log_score
            log_score -= max_log_score
            Z = np.sum(np.exp(log_score))
            pi = np.exp(log_score)/Z
            mem_pi += [pi]
            #for c in range(10):
            #    pi[c] = lik[c]/lik.sum()
            print 'pi =' + str(pi)
            print 'max(pi) = ',max(pi)
            sum_H = - pi * np.log(lik)
            #print np.log(lik)
            print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.where(pi == max(pi))[0][0]
            print '**** H = ',sum_H.sum()
    

In [ ]:
plt.figure(figsize = (12,6))
plt.semilogy(mem_pi)
plt.ylim([1e-30,10])
#plt.legend()

In [ ]:
def calc_lik(v,h,u):
    lik = np.zeros(10)
    for c in range(10):
        if np.linalg.norm(v) < 1e-16:
            if np.linalg.norm(mu[c][h][u]) > 1e-16:
                lik[c] = rho[c][h][u]
            else:
                lik[c] = 1                
        else:
            if np.linalg.norm(mu[c][h][u]) > 1e-16:
                dist = multivariate_normal(mean = mu[c][h][u], cov = Sigma[c][h][u])
                lik[c] = (1-rho[c][h][u]) * dist.pdf(v)
                #lik[c] =  dist.pdf(v)
            else:
                lik[c] = 0
        lik[c] = max(lik[c],1e-16)    
    return lik

def update_log_score(log_score, lik):
    #print 'lik =' + str(lik) 
    log_score += np.log(lik) 
    max_log_score = max(log_score)
    log_score -= max_log_score
    return log_score
    
def calc_pi(log_score): # TODO
    Z = np.sum(np.exp(log_score))
    pi = np.exp(log_score)/Z
    #print 'pi =' + str(pi)
    #print 'max(pi) = ',max(pi)
    return pi
    
def calc_H_plus(log_score, v_plus, h_plus, u_plus):
    lik_plus = calc_lik(v_plus, h_plus, u_plus)
    log_score_plus = update_log_score(np.copy(log_score), lik_plus)
    pi_plus = calc_pi(log_score_plus)
    sum_H = - pi_plus * np.log(pi_plus)
    #print 'sum_H : ', sum_H
    #print np.log(lik)
    #print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.where(pi == max(pi))[0][0]
    #print '**** H = ',
    return sum_H.sum()

def calc_F_plus(pi, log_score, v_plus, h_plus, u_plus):    
    lik_plus = calc_lik(v_plus, h_plus, u_plus)
    log_score_plus = update_log_score(np.copy(log_score), lik_plus)
    pi_plus = calc_pi(log_score_plus)
    sum_F = - pi_plus * (np.log(lik_plus) - np.log(pi_plus) + np.log(pi))
    #print 'sum_F : ', sum_F
    #print np.log(lik)
    #print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.where(pi == max(pi))[0][0]
    #print '**** H = ',
    return sum_F.sum()   

def calc_E_plus(pi, v_plus, h_plus, u_plus):    
    lik_plus = calc_lik(v_plus, h_plus, u_plus)
    sum_E = pi * lik_plus
    #print 'sum_F : ', sum_F
    #print np.log(lik)
    #print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.where(pi == max(pi))[0][0]
    #print '**** H = ',
    return sum_E.sum()   

### Test fonctions MAJ

In [ ]:
c_ref = 8
#h = 0
#u = (0,0)

for i in [0]:
    pi = np.ones(10) / 10
    log_score = np.zeros(10)
    mem_pi = [pi]
    mem_H = [np.sum(- pi * np.log(pi))]
    for h in range(-1,5):
        print '*************************'
        print '*********' , 'h = ' , h , '************'
        print '*************************'
        for u in U[h]:
            v = Data_test[c_ref][h][u][i]
            lik = calc_lik(v,h,u)
            print 'lik =' + str(lik) 
            log_score = update_log_score(log_score, lik)
            print 'log_score =' + str(log_score) 
            pi_moins = pi
            pi = calc_pi(log_score)
            mem_pi += [pi]
            print 'pi =' + str(pi)
            print 'max(pi) = ',max(pi)
            F = np.sum(- pi * (np.log(lik) - np.log(pi) + np.log(pi_moins)))
            H = np.sum(- pi * np.log(pi))
            mem_H += [H]
            #print np.log(lik)
            print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.where(pi == max(pi))[0][0]
            print '**** H = ', H
            print '**** F = ', F
    

In [ ]:
plt.figure(figsize = (20,4))
plt.subplot(1,3,1)
affiche_mnist(Data_test, c_ref, i)
plt.subplot(1,3,2)
plt.semilogy(mem_pi)
plt.ylim([1e-30,10])
plt.title('$\pi$')
plt.subplot(1,3,3)
plt.plot(mem_H)
plt.title('Entropy')


## Test reconnaissance en profondeur (saccade unique)

Tests entropie effective 

On constate un biais de confirmation de l'hypothèse initiale

In [ ]:
c_ref = 8
print '*****', c_ref, '*****'
#h = 0
#u = (0,0)

for i in [0]:
    pi = np.ones(10) / 10
    log_score = np.zeros(10)
    mem_pi = [pi]
    mem_pi_copy = []
    mem_H = [np.sum(- pi * np.log(pi))]
    mem_H_copy = []
    mem_h_u = []
    mem_out = []
    (h,u) = (-1,-1)
    v = Data_test[c_ref][h][u][i]     
    lik = calc_lik(v,h,u)
    print 'lik : ', lik
    log_score = update_log_score(log_score, lik)
    #print 'log_score : ', log_score
    pi = calc_pi(log_score)
    print 'pi : ', pi
    mem_pi += [pi]
    H = np.sum(- pi * np.log(pi))
    mem_H += [H]
    print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.where(pi == max(pi))[0][0]    
    
    (h, u) = (0, (0, 0))
    U_tree_ref = U_tree
    TOUR = 0
    END = False
    VU = False
    SEUIL = False
    liste_u = []
    mem_u_final = []
    while END == False:
        print h, u
        if (h,u) not in mem_h_u:
            mem_h_u += [(h,u)]
            v = Data_test[c_ref][h][u][i]     
            lik = calc_lik(v,h,u)
            print 'lik : ', lik
            log_score = update_log_score(log_score, lik)
            print 'log_score : ', log_score
            pi = calc_pi(log_score)
            print 'pi : ', pi
            mem_pi += [pi]
            H = np.sum(- pi * np.log(pi))
            print 'H : ', H
            mem_H += [H]
            print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.argmax(pi)
            out = np.argmax(pi)
            VU = False
            if h == 4 and pi[out] > .9999 :
                print pi[out]
                SEUIL = True
        else :
            print h, u, 'déjà vu'
            #VU = True
            print ''
        if h < 4:
            for i_fils in range(1,5):
                if U_tree_ref[i_fils] != [] and U_tree_ref[i_fils] not in liste_u and U_tree_ref[i_fils][0] not in mem_h_u:
                    liste_u+= [U_tree_ref[i_fils]] 
            print 'TOUR = ',TOUR, 'h = ', h, 'len(liste_u) =', len(liste_u)
            #c = raw_input()
            H_plus = np.zeros(len(liste_u))
            F_plus = np.zeros(len(liste_u))
            E_plus = np.zeros(len(liste_u))
            for k in range(len(liste_u)):
                if liste_u[k] != []:
                    (h_plus, u_plus) = liste_u[k][0]
                    if (h_plus, u_plus) not in mem_h_u :
                        #print 'hplus, u_plus :', (h_plus, u_plus)
                        v_plus = Data_test[c_ref][h_plus][u_plus][i]
                        H_plus[k] = calc_H_plus(log_score, v_plus, h_plus, u_plus)
                        F_plus[k] = calc_F_plus(pi, log_score, v_plus, h_plus, u_plus)
                        E_plus[k] = calc_E_plus(pi, v_plus, h_plus, u_plus)
                    else :
                        H_plus[k] = 9999999
                        F_plus[k] = -9999999                        
                        E_plus[k] = 9999999                        
                else:
                    H_plus[k] = 9999999
                    F_plus[k] = -9999999
                    E_plus[k] = 9999999                        

            #k_plus = np.argmin(H_plus) #where(H_plus == min(H_plus))[0][0]
            test = True
            while test:
                #k_plus =  np.argmax(F_plus) #np.where(F_plus == min(F_plus))[0][0]
                #k_plus = np.argmax(H_plus)
                k_plus = np.argmin(E_plus)
                h_plus = liste_u[k_plus][0][0]
                i_plus = liste_u[k_plus][0][1][0]
                i_moins = u[0]/2
                b_i_inf = 2**(h_plus-h+1) * i_moins
                b_i_sup = 2**(h_plus-h+1) * (i_moins + 1)
                j_plus = liste_u[k_plus][0][1][1]
                j_moins = u[1]/2
                b_j_inf = 2**(h_plus-h+1) * j_moins
                b_j_sup = 2**(h_plus-h+1) * (j_moins + 1)
                print 'i_plus :', i_plus, ' in [', b_i_inf, ',', b_i_sup, ']?', h
                print 'j_plus :', j_plus, ' in [', b_j_inf, ',', b_j_sup, ']?', h
                if h_plus > h and i_plus in range(b_i_inf,b_i_sup) and j_plus in range(b_j_inf,b_j_sup):
                    test = False              
                else:
                    F_plus[k_plus] = -9999999
                    H_plus[k_plus] = -9999999
                    E_plus[k_plus] = 9999999                
            print 'H plus :', H_plus
            print 'F plus :', F_plus
            print 'k_plus :', k_plus
            (h, u) = liste_u[k_plus][0]
            U_tree_ref = liste_u[k_plus] #U_tree_ref[k_plus + 1]
            #print U_tree_ref
            print ''
        else:
            mem_u_final += [u]
            mem_pi_copy += [list(mem_pi)]
            mem_H_copy += [list(mem_H)]
            mem_out += [out]
            if TOUR == 20  or SEUIL:    
                END = True
                print '************************************'
                print '******         FINI          *******'
                print '************************************'                
            else:
                TOUR += 1
                print '************************************'
                print '******       TOUR    ' + str(TOUR) + '        ******'
                print '************************************'
                h = 0
                u = (0,0)
                U_tree_ref = U_tree
        

In [ ]:
coeffs_aff = init_coeffs_mnist()
cpt = 0
path_i = []
path_j = []
for u in mem_u_final :
    path_i += [U[4][u][0]]
    path_j += [U[4][u][1]]
    plt.figure(figsize = (20,4))
    plt.subplot(1,4,1)
    affiche_mnist(Data_test, c_ref, i)
    affiche_path_mnist(path_i, path_j)
    plt.title(cpt)
    plt.subplot(1,4,2)    
    coord = U[4][u]  
    print 'coords :', coord
    
    affiche_mnist_coord(Data_test, coeffs_aff, c_ref, i, coord)
    affiche_path_mnist(path_i, path_j)
    plt.subplot(1,4,3)
    plt.plot(mem_pi_copy[cpt])
    plt.title('$\pi$, out = '+str(mem_out[cpt]))
    plt.ylim([0,1.05])
    plt.subplot(1,4,4)
    plt.plot(mem_H_copy[cpt])
    plt.title('Entropy')
    plt.ylim([0,2.4])
    cpt += 1

## Test en largeur d'abord (bottom up)

In [ ]:
U_tree[i_fils][1][0]

In [ ]:
mem_h_u

In [ ]:
U[4][(5,10)]

In [ ]:
lik

In [ ]:
np.log(lik)

In [ ]:
pi[out]

In [ ]:
lik_test = [6.47120236e+00,   1.00000000e-16,   5.20558153e-01,   1.37438396e-02,
   2.70854267e-05,   1.95244077e-01,   2.48980440e+00,   1.61746086e-04,
   3.05114329e-02,   1.48932662e-03]
log_score_test = [-44.72913926, -226.20581543,   -2.30103965,  -30.98412798,  -46.28824072,
  -32.42211479,  -30.40021047, -100.38374515,    0.,          -52.30459006]
print np.log(lik_test)

In [ ]:
log_score_plus = log_score_test + np.log(lik_test)

In [ ]:
log_score_plus

In [ ]:
b_i_inf = 2**(h-5) * i_moins

In [ ]:
2**(5-h) 

In [ ]:
h